In [ ]:
# David Data - Dataset processing (into multidimensional bins)

# Processing whole dataset as one, and seasons separately
# Data on normal lat-lon coordinates 

# Using 96 lat bins (so resulting data has the same number of bins as WACCM output used in magnetic latitude interpolation calculation)

# Original data has already filtered for Es. Where Es are detected, hmEs and s4s exist, else they are NaNs. So my processing says where not NaN, count as Es 
# Calculating Occurence freq over Lat-Lon only (as no altitude data where Es not detected)


In [29]:
import xarray as xr
import numpy as np
import cftime
import nc_time_axis
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib import ticker, cm
from cftime import datetime 
import datetime as dt
import matplotlib.colors as colors
import math
import random
import matplotlib.cm as mcm
from matplotlib.ticker import MaxNLocator
#jet = mcm.get_cmap('jet')
jet = mcm.get_cmap('jet') if isinstance(mcm.get_cmap('jet'), str) else mcm.get_cmap('jet')
import netCDF4 as nc
import sys
import os
import psutil
import netCDF4 as nc
import cartopy.crs as ccrs # CRS stands for "Coordinate reference systems" for map projection
from cartopy.crs import PlateCarree
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from dateutil import tz
import pytz
import time
from time import process_time
from tqdm import tqdm
import dask.array as da
import dask.dataframe as dd
import dask
%matplotlib inline 
import matplotlib.gridspec as gridspec
#import line_profiler
#%load_ext line_profiler

/tmp/ipykernel_501434/2282274303.py:16: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  jet = mcm.get_cmap('jet') if isinstance(mcm.get_cmap('jet'), str) else mcm.get_cmap('jet')


# Open WACCM dataset

In [30]:
################################################################################
# Open WACCM dataset to define bins
################################################################################

# arrays sliced between chosen range (~90-130km) indices 42-60, 19 levels

file1name='Nc_Files/SpE_Output/Wuhu_IonTr_run_SpE_Output_Dec-Feb.nc' 
wds = xr.open_dataset(file1name)

wds_lat = wds['lat']
wds_lon = wds['lon']

lev_sl = wds['lev_sl']
Zavg_sl = wds.variables['Zavg_sl'] 
 
altavg_sl = wds['altavg_sl']
# altavg_sl.values # array([131.1492157 , 126.73989868, 122.75083923, 119.16303253,
#        115.95503998, 113.10031128, 110.56510162, 108.3082428 ,
#        106.28289795, 104.44081879, 102.7361145 , 101.12830353,
#         99.58457184,  98.0800705 ,  96.59645081,  95.12115479,
#         93.64645386,  92.16801453,  90.6837616 ])


print('    Array lev = ' + str("%.1e" % lev_sl[-1] ) + 'hPa : ' + str("%.1e" % lev_sl[0] ) + 'hPa'
           + ' (Z3 approx ' + str("%.0f" % Zavg_sl[-1]) + '-' + str("%.0f" % Zavg_sl[0] ) + 'km' + ',' 
           + ' Alt approx ' + str("%.0f" % altavg_sl[-1]) + '-' + str("%.0f" % altavg_sl[0] ) + 'km' + ')' )

    Array lev = 1.3e-03hPa : 1.5e-05hPa (Z3 approx 89-129km, Alt approx 91-131km)


# Defining Lon Bins

In [31]:
# ################################################################################
# # Lon bins -180 -> 180
# ################################################################################

# Updated value for a range of -180 to +180 degrees
lon_upper_edge_last_bin = 180.0
lon_bin_edges = np.append(np.arange(-180.0, 180.0, 2.5), lon_upper_edge_last_bin)
lon_bin_midpoints = (lon_bin_edges[:-1] + lon_bin_edges[1:]) / 2

# Other calculations (width, number of bins, etc.) remain the same
lon_bin_widths = np.diff(lon_bin_edges)
lon_average_bin_width = np.mean(lon_bin_widths)
lon_num_bins = len(lon_bin_midpoints)

# # Print the results
# print("Lon Bin Edges (Lower and Upper):")
# for i in range(len(lon_bin_edges) - 1):
#     print(f"Bin {i + 1}: [{lon_bin_edges[i]}, {lon_bin_edges[i + 1]}]")

# print("\nLon Bin Midpoints:")
# for i in range(len(lon_bin_midpoints)):
#     print(f"Lon Bin {i + 1} Midpoint: {lon_bin_midpoints[i]}")

# print("\nLon Bin Widths:", lon_bin_widths)
# print(f"Average Lon Bin Width: {lon_average_bin_width:.8f} degrees")
# print(f"Number of Lon Bins: {lon_num_bins}")

# Defining lat bins for maglat interp

In [32]:
# if maglatbins == 1:
lat_bin_midpoints = wds['lat'].values
lat_bin_widths = np.diff(lat_bin_midpoints)

# Extend the array by one element with the same value as the last element
lat_bin_widths_ext = np.append(lat_bin_widths, lat_bin_widths[-1])

lat_bin_edges = np.zeros(lat_bin_midpoints.shape[0] + 1)
lat_bin_edges[0:-1] = lat_bin_midpoints - lat_bin_widths_ext/2
lat_bin_edges[-1] = lat_bin_midpoints[-1] + lat_bin_widths[-1]/2

#lat_average_bin_width = np.mean(lat_bin_widths)
lat_num_bins = len(lat_bin_midpoints)


# # ##Print the results
# print("Lat Bin Edges (Lower and Upper):")
# for i in range(len(lat_bin_edges) - 1):
#     print(f"Bin {i + 1}: [{lat_bin_edges[i]}, {lat_bin_edges[i + 1]}]")

# print("\nLat Bin Midpoints:")
# for i in range(len(lat_bin_midpoints)):
#     print(f"Bin {i + 1} Midpoint: {lat_bin_midpoints[i]}")

# print("\nLat Bin Widths:", lat_bin_widths)
# #print(f"Lat Average Bin Width: {lat_average_bin_width:.6f} degrees")
print(f"Lat Number of Bins: {lat_num_bins}")



Lat Number of Bins: 96


# Defining Altitude Bins

In [33]:
################################################################################
# Zavg_70150 bins - Note first bin is at top
# Bins between 70 and 150 km
################################################################################


wdss = xr.open_dataset('Nc_Files/ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport/ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport.cam.h2.0002-01-01-00000.nc')
geopH = geopH = wdss['Z3'] / 1000
Zavg = geopH.mean(('time','lat', 'lon'))

index_70 = np.abs(Zavg - 70).argmin()
index_150 = np.abs(Zavg - 150).argmin()

Zavg_70150 = Zavg[int(index_150.item()):int(index_70.item())+1]
#Zavg_70150


Zavg_70150_bin_widths = np.diff(Zavg_70150.values)
Zavg_70150_bin_edges = [Zavg_70150[0] - Zavg_70150_bin_widths[0] / 2] + [Zavg_70150[i] + Zavg_70150_bin_widths[i] / 2 for i in range(len(Zavg_70150_bin_widths))] + [Zavg_70150[-1] + Zavg_70150_bin_widths[-1] / 2]
Zavg_70150_bin_midpoints = [(Zavg_70150_bin_edges[i] + Zavg_70150_bin_edges[i + 1]) / 2 for i in range(len(Zavg_70150_bin_edges) - 1)]
Zavg_70150_num_bins = len(Zavg_70150_bin_midpoints)


# # Print the bin edges and midpoints
# print("Zavg_70150 Bin Edges (Lower and Upper):")
# for i in range(len(Zavg_70150_bin_edges) - 1):
#     print(f"Bin {i + 1}: [{Zavg_70150_bin_edges[i].values}, {Zavg_70150_bin_edges[i + 1].values}]")

# print("\nZavg_70150 Bin Midpoints:")
# for i in range(len(Zavg_70150_bin_midpoints)):
#     print(f"Bin {i + 1} Midpoint: {Zavg_70150_bin_midpoints[i].values}")

# # Print the bin widths
# print("\nZavg_70150 Bin Widths:")
# for i in range(len(Zavg_70150_bin_widths)):
#     print(f"Bin {i + 1} Width: {Zavg_70150_bin_widths[i]}")

# print(f"Number of Bins: {Zavg_70150_num_bins}")

# Load magnetic gridlines for plot

In [34]:
#Define magnetic latitude lines

filename='Nc_Files/ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport/ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport.cam.h0.0001-06.nc' 
ds = xr.open_dataset(filename)
ALATM = ds.variables['ALATM'] #Magnetic latitude at each geographic coordinate
ALat = ds.variables['lat']
ALon = ds.variables['lon']

# COSMIC Data Loading (Lat-Lon, 2006-2019)

In [35]:
cosmicnc = 'Nc_Files/s4max_files/COSMIC1_for_Leeds.h5'
cds = xr.open_dataset(cosmicnc)

Jdate = cds['Julian_date']
lat = cds['Latitude']
lon = cds['Longitude']
hmEs = cds['hmEs']
s4s = cds['s4s']

phony_dim_0 = cds['phony_dim_0']

s4s.shape #(7657632,)

(7657632,)

In [36]:
# Convert julian date to datetime
#======================================

datetimes = pd.to_datetime(Jdate.values, unit='D', origin='julian')

# datetime[0]  #Timestamp('2006-04-30 05:56:08.043825024')
# datetime[-1]  #Timestamp('2019-12-10 12:39:50.466977536')
datetimes.shape #(7657632,)

(7657632,)

In [37]:
# Print values of variables between a chosen range for info
#======================================

for index in range(1, 50):
    lat_value = lat[index].values
    lon_value = lon[index].values
    hmEs_value = hmEs[index].values
    s4s_value = s4s[index].values
    datetime_value = datetimes[index]
    Jdate_value = Jdate[index].values

    #print(f"Index {index}: dt = {datetime_value}, lat = {lat_value}, lon = {lon_value}, hmEs = {hmEs_value}, s4s = {s4s_value}")

In [38]:
# Sort into consecutive time order
#======================================


cds_sorted = cds.sortby('Julian_date')

# Extract sorted variables
sorted_datetimes = pd.to_datetime(cds_sorted['Julian_date'].values, unit='D', origin='julian')
sorted_lat = cds_sorted['Latitude'].values
sorted_lon = cds_sorted['Longitude'].values
sorted_hmEs = cds_sorted['hmEs'].values
sorted_s4s = cds_sorted['s4s'].values


# # Print a subset of sorted variables for indices 0 to 99
# for i in range(50):
#     print(f"Ind {i}: dt = {sorted_datetimes[i]}, lat = {sorted_lat[i]}, lon = {sorted_lon[i]}, hmEs = {sorted_hmEs[i]}, s4s = {sorted_s4s[i]}")

In [39]:
# Identify indices where date/time = nan
# Remove occultations where date/time = nan
#======================================

ind_to_remove = []

for index in range(len(sorted_datetimes)):
    if pd.isna(sorted_datetimes[index]):
        lat_value = sorted_lat[index]
        lon_value = sorted_lon[index]
        hmEs_value = sorted_hmEs[index]
        s4s_value = sorted_s4s[index]
        
        ind_to_remove.append(index)
        
        #print(f"Index {index}: lat = {lat_value}, lon = {lon_value}, hmEs = {hmEs_value}, s4s = {s4s_value}")

## Create a new DataFrame without the specified indices
filtered_df = pd.DataFrame({'datetime': sorted_datetimes, 'lat': sorted_lat, 'lon': sorted_lon, 'hmEs': sorted_hmEs, 's4s': sorted_s4s})

# Use the separate variable ind_to_remove to remove indices
filtered_df = filtered_df[~filtered_df.index.isin(ind_to_remove)]

# Print the resulting DataFrame
print(filtered_df)

                             datetime        lat         lon  hmEs  s4s
0       2006-04-30 05:56:08.043825024  51.270003   95.375549   NaN  NaN
1       2006-04-30 05:56:36.604321920  64.094526  123.877350   NaN  NaN
2       2006-04-30 05:56:53.465184000  21.259120  137.635163   NaN  NaN
3       2006-04-30 06:01:05.802982400  45.922146  119.382021   NaN  NaN
4       2006-04-30 06:03:08.944861312  35.060482  143.047881   NaN  NaN
...                               ...        ...         ...   ...  ...
7657516 2019-12-10 11:30:26.465624576  23.987218   78.103606   NaN  NaN
7657517 2019-12-10 11:39:52.366115840 -11.536251   79.228695   NaN  NaN
7657518 2019-12-10 11:48:34.206978560 -49.524546   64.884119   NaN  NaN
7657519 2019-12-10 12:39:28.187154944  49.982191 -112.757110   NaN  NaN
7657520 2019-12-10 12:39:50.466977536  43.841069  -96.843928   NaN  NaN

[7657521 rows x 5 columns]


In [40]:
# Load sliced data and re index so consecutive numbers
#======================================

fdatetimes = filtered_df['datetime']
flat = filtered_df['lat']
flon = filtered_df['lon']
fhmEs = filtered_df['hmEs']
fs4s = filtered_df['s4s']

# Reset indices for consistency
fdatetimes.reset_index(drop=True, inplace=True)
flat.reset_index(drop=True, inplace=True)
flon.reset_index(drop=True, inplace=True)
fs4s.reset_index(drop=True, inplace=True)

# COSMIC Data Processing: Occ Freq (lat-**lon**), Whole Dataset

In [41]:
# Processing normal Lat Lon coordinates 
# Whole Dataset   
#======================================

averaging_period = 'Whole_Dataset'

freq_lon = np.zeros((lat_num_bins, lon_num_bins))
nmeas_lon = np.zeros((lat_num_bins, lon_num_bins))
ocfr_lon = np.zeros((lat_num_bins, lon_num_bins))    
 
ds_length = np.arange(fdatetimes.shape[0])

for i in tqdm(ds_length):
    current_lat = flat.values[i]
    current_lon = flon.values[i]
    current_s4max = fs4s.values[i]
    current_time = fdatetimes[i]
    lat_bin_index = np.digitize(current_lat, lat_bin_edges) - 1
    lon_bin_index = np.digitize(current_lon, lon_bin_edges) - 1

    nmeas_lon[lat_bin_index, lon_bin_index] += 1.

    if not np.isnan(current_s4max):
        freq_lon[lat_bin_index, lon_bin_index] += 1.

ocfr_lon = (freq_lon / nmeas_lon) * 100.0

###########################################################################################################################
# Save occurrence frequency to nc file
Ocfr_ds = xr.Dataset(
    data_vars={
        "ocfr_lon": (['latitude', 'longitude'], ocfr_lon),
    },
    coords={
        #"altitude": Zavg_70150_bin_midpoints,
        "latitude": lat_bin_midpoints,
        "longitude": lon_bin_midpoints,
    },
    attrs={
        'averaging_period': averaging_period,
    }
)

output_directory = "./Nc_Files/s4max/Daviddata/"

output_file = f"{output_directory}Ocfr_Daviddata_{averaging_period}_ML.nc"

Ocfr_ds.to_netcdf(output_file)

###########################################################################################################################


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7657521/7657521 [02:05<00:00, 61071.17it/s]


# COSMIC Data Processing: Occ Freq (lat-**lon**), Seasons Separately

In [27]:
# Processing Normal Lat Lon coordinates
# Seasons separately
#======================================

# Define multiple sets of desired months
all_desired_months = [ [3, 4, 5] , [6, 7, 8], [9, 10, 11] , [12, 1, 2] ]

ds_length = np.arange(fdatetimes.shape[0])


# Loop over each set of desired months
for desired_months in all_desired_months:
    print('Months ' + str(desired_months))

    season = ""
    if set(desired_months) <= {12, 1, 2}:
        season = "winter"
    elif set(desired_months) <= {3, 4, 5}:
        season = "spring"
    elif set(desired_months) <= {6, 7, 8}:
        season = "summer"
    elif set(desired_months) <= {9, 10, 11}:
        season = "autumn"
    else:
        season = "unknown"
        
    print(season)
    
    averaging_period = 'Three-Month'
    
    freq_lon = np.zeros((lat_num_bins, lon_num_bins))
    nmeas_lon = np.zeros((lat_num_bins, lon_num_bins))
    ocfr_lon = np.zeros((lat_num_bins, lon_num_bins))    
 
    for i in tqdm(ds_length):
        current_lat = flat.values[i]
        current_lon = flon.values[i]
        current_s4max = fs4s.values[i]
        current_time = fdatetimes[i]
        current_month = current_time.month
        
        if current_month in desired_months:      
            lat_bin_index = np.digitize(current_lat, lat_bin_edges) - 1
            lon_bin_index = np.digitize(current_lon, lon_bin_edges) - 1

            nmeas_lon[lat_bin_index, lon_bin_index] += 1.

            if not np.isnan(current_s4max):
                freq_lon[lat_bin_index, lon_bin_index] += 1.

    ocfr_lon = (freq_lon / nmeas_lon) * 100.0

    ###########################################################################################################################
    # Save occurrence frequency to nc file
    Ocfr_ds = xr.Dataset(
        data_vars={
            "ocfr_lon": (['latitude', 'longitude'], ocfr_lon),
        },
        coords={
            "latitude": lat_bin_midpoints,
            "longitude": lon_bin_midpoints,
        },
        attrs={
            'averaging_period': averaging_period,
        }
    )

    output_directory = "./Nc_Files/s4max/Daviddata/"

    output_file = f"{output_directory}Ocfr_Daviddata_{season}_ML.nc"
    
    Ocfr_ds.to_netcdf(output_file)

    ###########################################################################################################################


Months [3, 4, 5]
spring


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7657521/7657521 [01:09<00:00, 110548.50it/s]
/tmp/ipykernel_501434/2066068107.py:51: RuntimeWarning: invalid value encountered in divide
  ocfr_lon = (freq_lon / nmeas_lon) * 100.0


Months [6, 7, 8]
summer


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7657521/7657521 [01:09<00:00, 110439.27it/s]


Months [9, 10, 11]
autumn


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7657521/7657521 [01:08<00:00, 111986.20it/s]


Months [12, 1, 2]
winter


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7657521/7657521 [01:08<00:00, 111437.50it/s]
